In [8]:
class Apriori:
    def __init__(self,dataset="tt",min_support=0.5,min_confidence=0.5,min_lift=0.5,min_length=1):
        self.min_support    = min_support
        self.min_confidence = min_confidence
        self.min_lift       = min_lift
        self.min_length     = min_length
        self.dataset        = dataset
        self.confidences    = self.run_confidence()
        self.support        = self.run_support()

    # calculate the frequency of items 
    # takes list of items
    def calculate_frequency(self,items):
        count = 0
        if len(items) == 1:
            count = sum([count+1 for row in self.dataset if items[0] in row])
        if len(items) == 2:
            count = sum([count+1 for row in self.dataset if (items[0] in row) and (items[1] in row)])
        return count
    
    # Calculate Support of items
    # takes list of items
    def calculate_support(self,items):
        top    = self.calculate_frequency(items)
        N      = len(self.dataset) # The number of transactions
        return top/N
    
    # Calculate the confidence 
    # Takes list of items
    def calculate_confidence(self,items):
        top    = self.calculate_frequency(items)
        bottom = self.calculate_frequency([items[0]]) 
        return top/bottom
    
    def calculate_lift(self,items):
        top = self.calculate_support(items)
        bottom = self.calculate_support([items[0]]) * self.calculate_support([items[1]])
        return top / bottom
    
    def run_confidence(self):
        confidances = {}
        all_items = list(set([i for row in self.dataset for i in row ]))
        for i in all_items:
            for j in all_items:
                if i not in confidances.keys():
                    confidances[i] = []
                confidances[i].append(self.calculate_confidence([i,j]))
        
        confidances = pd.DataFrame(confidances,index=all_items).T
        return confidances
    def run_support(self):
        support = {}
        all_items = list(set([i for row in self.dataset for i in row ]))
        for i in all_items:
            for j in all_items:
                if i not in support.keys():
                    support[i] = []
                support[i].append(self.calculate_support([i,j]))
        
        support = pd.DataFrame(support,index=all_items).T
        return support

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/dataset.csv",names = ['id','transaction'])
df.head()

,id,transaction
0,1,Milk- Onion- Nutmeg- KidneyBeans- Eggs- Yogurt
1,2,Dill- Onion- Nutmeg- KidneyBeans- Eggs- Yogurt
2,3,Milk- Apple- Eggs- Yogurt
3,4,Milk- Unicorn- Corn- Onion- Yogurt
4,5,Corn- Onion- Yogurt- KidneyBeans- Icecream- Eggs


In [3]:
data = list(df['transaction'].apply(lambda x: x.replace(" ","").split("-")))
data

[['Milk', 'Onion', 'Nutmeg', 'KidneyBeans', 'Eggs', 'Yogurt'],
 ['Dill', 'Onion', 'Nutmeg', 'KidneyBeans', 'Eggs', 'Yogurt'],
 ['Milk', 'Apple', 'Eggs', 'Yogurt'],
 ['Milk', 'Unicorn', 'Corn', 'Onion', 'Yogurt'],
 ['Corn', 'Onion', 'Yogurt', 'KidneyBeans', 'Icecream', 'Eggs']]

# MLXTEND

In [4]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(data)


In [5]:
dd=te_ary.transform(data)

In [6]:
dd

array([[False, False, False,  True, False,  True,  True,  True,  True,
        False,  True],
       [False, False,  True,  True, False,  True, False,  True,  True,
        False,  True],
       [ True, False, False,  True, False, False,  True, False, False,
        False,  True],
       [False,  True, False, False, False, False,  True, False,  True,
         True,  True],
       [False,  True, False,  True,  True,  True, False, False,  True,
        False,  True]])

In [7]:
te.columns_

['Apple',
 'Corn',
 'Dill',
 'Eggs',
 'Icecream',
 'KidneyBeans',
 'Milk',
 'Nutmeg',
 'Onion',
 'Unicorn',
 'Yogurt']

In [10]:
df = pd.DataFrame(dd, columns=te.columns_)
df

,Apple,Corn,Dill,Eggs,Icecream,KidneyBeans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,False,True,False,False,False,True
3,False,True,False,False,False,False,True,False,True,True,True
4,False,True,False,True,True,True,False,False,True,False,True


In [11]:
from mlxtend.frequent_patterns import apriori
#apriori(df, min_support\=0.6)

In [16]:
apriori(df, min_support=0.5,use_colnames=True)


,support,itemsets
0,0.8,(Eggs)
1,0.6,(KidneyBeans)
2,0.6,(Milk)
3,0.8,(Onion)
4,1.0,(Yogurt)
5,0.6,"(KidneyBeans, Eggs)"
6,0.6,"(Onion, Eggs)"
7,0.8,"(Yogurt, Eggs)"
8,0.6,"(KidneyBeans, Onion)"
9,0.6,"(KidneyBeans, Yogurt)"


In [18]:
frequent_itemsets = apriori(df, min_support=0.6,use_colnames=True)
frequent_itemsets['frequency'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,frequency
0,0.8,(Eggs),1
1,0.6,(KidneyBeans),1
2,0.6,(Milk),1
3,0.8,(Onion),1
4,1.0,(Yogurt),1
5,0.6,"(KidneyBeans, Eggs)",2
6,0.6,"(Onion, Eggs)",2
7,0.8,"(Yogurt, Eggs)",2
8,0.6,"(KidneyBeans, Onion)",2
9,0.6,"(KidneyBeans, Yogurt)",2


In [19]:
    frequent_itemsets[ (frequent_itemsets['frequency'] ==2) & (frequent_itemsets['support'] >= 0.8) ]

,support,itemsets,frequency
7,0.8,"(Yogurt, Eggs)",2
11,0.8,"(Onion, Yogurt)",2


In [21]:
frequent_itemsets[ frequent_itemsets['itemsets'] == {'Onion', 'Eggs'} ]

,support,itemsets,frequency
6,0.6,"(Onion, Eggs)",2


In [ ]:
from sklearn